# Transforming Lindbladian equations to Krauss operators
sources: 
- https://arxiv.org/abs/1412.5746
- https://github.com/rigetti/forest-benchmarking/blob/4c2c3bf94af4926b61e9072ca71b914972de338c/forest/benchmarking/operator_tools/superoperator_transformations.py#L1
- https://forest-benchmarking.readthedocs.io/en/latest/superoperator_representations.html


# Lindbladian
### $\frac{d{\rho}}{dt} = -i[{H},{\rho}] + \sum_{j=1}^N \left( {L}_j {\rho} {L}_j^\dagger - \frac{1}{2} {L}_j^\dagger {L}_j {\rho} - \frac{1}{2} {\rho} {L}_j^\dagger {L}_j \right)$

where:

- $\frac{d{\rho}}{dt}$ is the time derivative of the density operator ${\rho}$
- ${H}$ is the Hamiltonian operator of the system
- ${L}_j$ are the Lindblad operators
- $N$ is the number of Lindblad operators
- $\dagger$ denotes the Hermitian conjugate of an operator

# Superoperator (Liouville)

### $\mathcal{L} =  -i\, H \otimes I + i  \otimes {H}^T + \sum_{j} \left( L_{j} \otimes L_j^* - \frac{1}{2} (L^\dagger_j L_j)\otimes I  - \frac{1}{2}\; I \otimes (L^T_j L_j^*) \right)$

Which acts on the density matrix as:

### $|\rho(t)\rangle \rangle = e^{\tau \mathcal L } |\rho\rangle\rangle$
### $\hat{\mathcal{E}} = |  e^{\tau \mathcal L } \rangle\rangle$

# Kraus Channel

### $\mathcal{E}(\rho(0)) = \sum_k A_k \rho(0) A^\dagger_k = \rho(t)$ 

# Choi-Jamiolkowski Isomorphism

\begin{split}\begin{align*}
\mathcal C &= I\otimes \mathcal E (|\phi^\dagger \rangle \langle \phi^\dagger|) \\\\
&=\sum_i (A_i \otimes I) |\phi^\dagger \rangle \langle \phi^\dagger  | ( A_i^\dagger \otimes I)\\\\
& = \frac{1}{d} \sum_i {\rm vec}(A_i)  {\rm vec} (A_i) ^\dagger \\\\
& = \frac{1}{d} \sum_i |A_i\rangle \rangle \langle\langle A_i |.
\end{align*}\end{split}

## Reminder:
Order of legs: (output), (input) = (rows), (columns) (from up to down in a diagram)

# Step 1: 
get the hamiltonian and jump operators as a list

In [43]:
import sys
sys.path.append('..')
# reload local packages automatically
%load_ext autoreload
%autoreload 2
from opentn.states.qubits import get_ladder_operator, I
import numpy as np
from scipy.linalg import expm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
# get Hamilontian
H = []

# NOTE: the gamma used in ADC is different than the one from the lindbladian
# Relation: gamma_adc = 1 - exp(-gamma_lind*t) <-> gamma_lind = -ln(1-gamma_adc)/t
gamma_adc = 0.5
tao = 1
gamma_lind = -np.log(1-gamma_adc)/tao


d = 2
L1 = np.sqrt(gamma_lind)*get_ladder_operator() # sigma-
# L2 = np.sqrt(gamma)*get_ladder_operator(adjoint=True) #sigma+
Li = [L1]
# Now here I need to create a structure that converts the thins to superoperator (maybe first save them as lindbladian)
# TODO: generalize so that we don't have only the same level. Here d=2 everywhere
super = np.zeros(shape=(d ** 2, d ** 2),dtype=complex)
# if H:
#     super = np.kron()
for L in Li:
    super += np.kron(L, L.conj()) - 0.5*np.kron(L.T.conj()@L, I) - 0.5*np.kron(I, L.T@L.conj())
super

array([[ 0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.69314718+0.j],
       [ 0.        +0.j, -0.34657359+0.j,  0.        +0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.        +0.j, -0.34657359+0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.69314718+0.j]])

In [72]:
from opentn.transformations import vectorize
test = vectorize(L1)
test

array([0.        +0.j, 0.83255461+0.j, 0.        +0.j, 0.        +0.j])

# Step 2:
get exponetial and convert to choi matrix


In [45]:
super_exp = expm(super*tao)
print(super_exp)
# we reshape the thing into tensor of 4 legs
choi = np.reshape(super_exp, [d] * 4)
choi = choi.swapaxes(1, 2).reshape([d ** 2, d ** 2]) #see graphical proof
choi

[[1.        +0.j 0.        +0.j 0.        +0.j 0.5       +0.j]
 [0.        +0.j 0.70710678+0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.70710678+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 0.5       +0.j]]


array([[1.        +0.j, 0.        +0.j, 0.        +0.j, 0.70710678+0.j],
       [0.        +0.j, 0.5       +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.70710678+0.j, 0.        +0.j, 0.        +0.j, 0.5       +0.j]])

# Step 3:
get krauss operators using the cholesky decomposition

# Cholesky: Error
`LinAlgError: Matrix is not positive definite`

In [62]:
# A_matrix = np.linalg.cholesky(choi)
# A_matrix

# Eigendecomposition: Correct

In [66]:
# krauss operators unvectorized from eigenvector with eigenvalue scaled
from opentn.transformations import unvectorize
# source: https://stackoverflow.com/questions/5086789/python-is-there-an-inverse-for-ndarray-flattenf

# try the eigenvector way
eigvals, eigvecs = np.linalg.eigh(choi)

tol = 1e-9
krauss_list = [np.sqrt(eigval) * unvectorize(np.array(v)) for eigval, v in
            zip(eigvals, eigvecs.T) if abs(eigval) > tol]
krauss_list

[array([[ 0.        +0.j, -0.70710678+0.j],
        [ 0.        +0.j,  0.        +0.j]]),
 array([[-1.        +0.j,  0.        +0.j],
        [ 0.        +0.j, -0.70710678+0.j]])]

In [50]:
a = np.array([[1,2],[3,4],[5,6]])
a.shape

(3, 2)

# Full workflow incorporated

In [75]:
from opentn.transformations import lindbladian2kraus
lindbladian2kraus(Li=[L1])

[array([[ 0.        +0.j, -0.70710678+0.j],
        [ 0.        +0.j,  0.        +0.j]]),
 array([[-1.        +0.j,  0.        +0.j],
        [ 0.        +0.j, -0.70710678+0.j]])]